In [ ]:
import pandas as pd
import statsmodels.api as sm
from google.colab import files
import numpy as np

# We chose to upload data set manually everytime we run the code, since
#we ran into some issues while uploading the Star_data set manually into Colab.
uploaded = files.upload()
file_path = list(uploaded.keys())[0]
data_star = pd.read_excel(file_path)
#2(a)
def print_exercise_header(exercise_number):
    print("\n" * 2)  #
    print("2A")
    print("-------------------------------------------------------------------")
    print("\n")

print_exercise_header("2(A)")

#Here we renamed collumns from the original data set.
data_star.rename(columns={
    'tscorek': 'test_score_k',
    'tscore1': 'test_score_1',
    'tscore2': 'test_score_2',
    'tscore3': 'test_score_3',
    'sck': 'small_class_k',
    'sc1': 'small_class_1',
    'sc2': 'small_class_2',
    'sc3': 'small_class_3',
    'rak': 'regular_aide_k',
    'ra1': 'regular_aide_1',
    'ra2': 'regular_aide_2',
    'ra3': 'regular_aide_3',
    'totexpk': 'teacher_exp_k',
    'boy': 'boy',
    'black': 'black',
    'freelunk': 'free_lunch_k'
}, inplace=True)

# Function for table 13.1
def prepare_data_for_grade(data_star, grade):
    """Prepares data for a specific grade."""
    return data_star[[f'test_score_{grade}', f'small_class_{grade}', f'regular_aide_{grade}', f'schid{grade}n']].dropna().rename(
        columns={
            f'test_score_{grade}': 'test_score',
            f'small_class_{grade}': 'small_class',
            f'regular_aide_{grade}': 'regular_aide',
            f'schid{grade}n': 'school_id'
        }
    )

# Table 13.1 generated with standard errors
grades = ['k', '1', '2', '3']
results_table_13_1 = []

for grade in grades:
    grade_data = prepare_data_for_grade(data_star, grade)
    X = grade_data[['small_class', 'regular_aide']].astype(float)
    X = sm.add_constant(X)
    y = grade_data['test_score'].astype(float)

    model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': grade_data['school_id'].astype(int)})
    coef, se, conf_int = model.params, model.bse, model.conf_int(alpha=0.05)

    # Finding the  confidence intervals for 'small_class' and 'regular_aide'
    small_class_ci_lower, small_class_ci_upper = conf_int.loc['small_class']
    regular_aide_ci_lower, regular_aide_ci_upper = conf_int.loc['regular_aide']

    results_table_13_1.append({
        'Grade': grade.upper(),
        'Small Class': f"{coef['small_class']:.2f} ({se['small_class']:.2f}) [{small_class_ci_lower:.2f}, {small_class_ci_upper:.2f}]",
        'Regular Aide': f"{coef['regular_aide']:.2f} ({se['regular_aide']:.2f}) [{regular_aide_ci_lower:.2f}, {regular_aide_ci_upper:.2f}]",
        'Intercept': f"{coef['const']:.2f} ({se['const']:.2f})",
        'N': len(grade_data)
    })

table_13_1 = pd.DataFrame(results_table_13_1)

print("Table 13.1: Differences Estimates of Effect on Standardized Test Scores of Class Size Treatment Group")
print(table_13_1.to_string(index=False))

table_13_1_file = "table_13_1_results.xlsx"
table_13_1.to_excel(table_13_1_file, index=False)
#Generating table 13.2
import pandas as pd
import statsmodels.api as sm
import numpy as np

# Table 13.2 Summary Table
table_13_2_summary_table = []

# Define the models and the variables they include
columns = [
    ['small_class', 'regular_aide'],
    ['small_class', 'regular_aide', 'teacher_exp'],
    ['small_class', 'regular_aide', 'teacher_exp'],
    ['small_class', 'regular_aide', 'teacher_exp', 'boy', 'free_lunch', 'black', 'race_other']
]

kindergarten_data = data_star[['test_score_k', 'small_class_k', 'regular_aide_k', 'teacher_exp_k', 'boy', 'black', 'free_lunch_k', 'schidkn', 'other']].rename(
    columns={
        'test_score_k': 'test_score',
        'small_class_k': 'small_class',
        'regular_aide_k': 'regular_aide',
        'teacher_exp_k': 'teacher_exp',
        'free_lunch_k': 'free_lunch',
        'schidkn': 'school_id',
        'other': 'race_other'
    }
)

for i, col in enumerate(columns):
    # Filter the dataset for the current model's columns + test_score and school_id
    relevant_columns = ['test_score', 'school_id'] + col
    df = kindergarten_data[relevant_columns].dropna()

    if i >= 2:
        school_dummies = pd.get_dummies(df['school_id'], prefix='school')
        X = pd.concat([df[col], school_dummies], axis=1).astype(float)
        school_indicator_variables = "yes"
    else:
        X = df[col].astype(float)
        school_indicator_variables = "no"

    # Dynamically decide whether to add intercept
    if i < 2:  # For models 1 and 2, include intercept
        X = sm.add_constant(X)
    else:  # For models 3 and 4, check if the constant (intercept) exists in X
        if 'const' not in X.columns:
            X = sm.add_constant(X)

    y = df['test_score'].astype(float)

    # Fit the model with clustered standard errors
    model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['school_id'].astype(int)})
    coef, se = model.params, model.bse

    # Create a summary entry for the current model
    model_summary = {
        'Model': str(i + 1),
        'Small Class Coef': f"{coef.get('small_class', np.nan):.2f} ({se.get('small_class', np.nan):.2f})",
        'Regular Aide Coef': f"{coef.get('regular_aide', np.nan):.2f} ({se.get('regular_aide', np.nan):.2f})"
    }

    # Add the intercept if it's present in the model
    if 'const' in coef:
        model_summary['Intercept'] = f"{coef.get('const', np.nan):.2f} ({se.get('const', np.nan):.2f})"
    else:
        model_summary['Intercept'] = np.nan

    # Add additional coefficients for models that include these variables
    for var in ['teacher_exp', 'boy', 'free_lunch', 'black', 'race_other']:
        if var in col:
            model_summary[f"{var.capitalize()} Coef"] = f"{coef.get(var, np.nan):.2f} ({se.get(var, np.nan):.2f})"
        else:
            model_summary[f"{var.capitalize()} Coef"] = np.nan

    # R-squared and number of observations
    model_summary['R-squared'] = round(model.rsquared, 2)
    model_summary['Number of observations'] = len(df)

    # Append the school indicator variable information
    model_summary['School indicator variables?'] = school_indicator_variables

    # Append the summary to the results list
    table_13_2_summary_table.append(model_summary)

# Create DataFrame for Table 13.2
table_13_2 = pd.DataFrame(table_13_2_summary_table)

# Print Table 13.2 in the desired format
print("\nTable 13.2: Differences Estimates with Additional Regressors for Kindergarten")
print(table_13_2.to_string(index=False))

# Save Table 13.2 to Excel
table_13_2_file = "table_13_2_results.xlsx"
table_13_2.to_excel(table_13_2_file, index=False)
# 2(b): Randomization Test for Kindergarten
def print_exercise_header(exercise_number):
    print("\n" * 2)
    print("2B")
    print("-------------------------------------------------------------------")
    print("\n")

print_exercise_header("2(B)")
import pandas as pd
import statsmodels.api as sm

# List to hold results of the randomization tests
randomization_results = []

# List of covariates for the randomization test
covariates = ['boy', 'free_lunch_k', 'white']

# Iterate over the covariates to perform the randomization test
for cov in covariates:
    # Selecting relevant columns and dropping any missing values
    dataset_subset = data_star[['small_class_1', 'regular_aide_1', cov]].dropna()

    # Define the dependent variable (covariate) and independent variables
    y_var = dataset_subset[cov].astype(float)
    X_vars = dataset_subset[['small_class_1', 'regular_aide_1']].astype(float)
    X_vars = sm.add_constant(X_vars)  # Add a constant for the regression

    # Fit the Ordinary Least Squares (OLS) regression model
    regression_model = sm.OLS(y_var, X_vars).fit()

    # Conduct F-test to determine if both coefficients are jointly zero
    f_test_results = regression_model.f_test("small_class_1 = regular_aide_1 = 0")

    # Store the results for this covariate
    randomization_results.append({
        'Covariate': cov,
        'Small Class Estimate': regression_model.params.get('small_class_1', float('nan')),
        'Regular Aide Estimate': regression_model.params.get('regular_aide_1', float('nan')),
        'F-statistic': float(f_test_results.fvalue),
        'P-value': float(f_test_results.pvalue)
    })

# Create a DataFrame from the results list
randomization_summary_df = pd.DataFrame(randomization_results)

# Display the summary without lines or extra formatting
print("\nRandomization Test")
for idx, row in randomization_summary_df.iterrows():
    print(f"Covariate: {row['Covariate']}")
    print(f"  Small Class Estimate: {row['Small Class Estimate']:.3f}")
    print(f"  Regular Aide Estimate: {row['Regular Aide Estimate']:.3f}")
    print(f"  F-statistic: {row['F-statistic']:.3f}")
    print(f"  P-value: {row['P-value']:.3e}")

# Save results to an Excel file for further review
output_file_updated = "randomization_test_results.xlsx"
randomization_summary_df.to_excel(output_file_updated, index=False)
print("The results suggest that randomization was well conducted for the covariates boy and")
print("free_lunch_k, as the F-tests show no significant differences across groups. ")
print("However, for the covariate white, the F-test indicates a significant imbalance, suggesting ")
print("that randomization was not fully successful for this variable.  ")
print("This result highlights the potential need to control for race in the analysis to ensure unbiased estimates.")
# 2(c): Attrition Analysis from Kindergarten to Grade 3
def print_exercise_header(exercise_number):
    print("\n" * 2)
    print(f"{exercise_number}")
    print("-------------------------------------------------------------------")
    print("\n")


print_exercise_header("2(C)")

import pandas as pd
import numpy as np

def get_k_condition(row):
    """Determine kindergarten condition"""
    if row['small_class_k'] == 1:
        return 'small'
    elif row['regular_aide_k'] == 1:
        return 'aide'
    else:
        return 'regular'

def get_g3_condition(row):
    """Determine grade 3 condition"""
    if row['small_class_3'] == 1:
        return 'small'
    elif row['regular_aide_3'] == 1:
        return 'aide'
    else:
        return 'regular'

# Add condition columns
data_star['k_condition'] = data_star.apply(get_k_condition, axis=1)
data_star['g3_condition'] = data_star.apply(get_g3_condition, axis=1)

# Function to check if student has valid data
def has_valid_data(row):
    """Check if student has valid test scores or condition data"""
    return pd.notna(row['treadss3']) or pd.notna(row['tmathss3'])

# Calculate retention for each condition
conditions = ['small', 'aide', 'regular']
results = {}

for condition in conditions:
    # Get all students who started in this condition
    k_students = data_star[data_star['k_condition'] == condition]
    initial_count = len(k_students)

    # Get students who stayed in same condition and have valid data
    stayed = k_students[
        (k_students['g3_condition'] == condition) &
        k_students.apply(has_valid_data, axis=1)
    ]
    stayed_count = len(stayed)

    # Calculate retention rate
    retention = stayed_count / initial_count if initial_count > 0 else 0

    results[condition] = {
        'initial': initial_count,
        'stayed': stayed_count,
        'retention': retention
    }

# Print results in a tabular format
print("\nDetailed Results:")
print("-------------------------------------------------------------------------------")
print(f"{'Condition':<15}{'Started in K':<15}{'Stayed through G3':<20}{'Retention Rate':<15}")
print("-------------------------------------------------------------------------------")
for condition in conditions:
    print(f"{condition.capitalize():<15}{results[condition]['initial']:<15}{results[condition]['stayed']:<20}{results[condition]['retention']:<15.3f}")
print("\n")

# Additional analysis: Movement Analysis
print("Movement Analysis:")
print("-------------------------------------------------------------------------------")
print(f"{'Starting Condition':<20}{'To Small':<15}{'To Aide':<15}{'To Regular':<15}")
print("-------------------------------------------------------------------------------")

for k_cond in conditions:
    k_students = data_star[data_star['k_condition'] == k_cond]
    total = len(k_students)

    to_small = len(k_students[k_students['g3_condition'] == 'small'])
    to_aide = len(k_students[k_students['g3_condition'] == 'aide'])
    to_regular = len(k_students[k_students['g3_condition'] == 'regular'])

    print(f"{k_cond.capitalize():<20}{to_small:<15}({to_small / total * 100:.1f}%)"
          f"{to_aide:<15}({to_aide / total * 100:.1f}%)"
          f"{to_regular:<15}({to_regular / total * 100:.1f}%)")



#2(d)
import pandas as pd
import statsmodels.api as sm
import numpy as np

def print_exercise_header(exercise_number):
    print("\n" * 2)
    print(f"{exercise_number}")
    print("-------------------------------------------------------------------")
    print("\n")
print_exercise_header("2(D)")

# Restrict data to students present from Kindergarten to Grade 3
restricted_data = data_star[
    data_star['test_score_k'].notna() &
    data_star['test_score_1'].notna() &
    data_star['test_score_2'].notna() &
    data_star['test_score_3'].notna()
]

# Prepare data for Table 13.1 with the restricted sample
grades = ['k', '1', '2', '3']
restricted_results_table_13_1 = []

for grade in grades:
    grade_data = restricted_data[[f'test_score_{grade}', f'small_class_{grade}', f'regular_aide_{grade}', f'schid{grade}n']].dropna().rename(
        columns={
            f'test_score_{grade}': 'test_score',
            f'small_class_{grade}': 'small_class',
            f'regular_aide_{grade}': 'regular_aide',
            f'schid{grade}n': 'school_id'
        }
    )

    X = grade_data[['small_class', 'regular_aide']].astype(float)
    X = sm.add_constant(X)
    y = grade_data['test_score'].astype(float)

    # OLS regression model with clustering standard errors by school_id
    model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': grade_data['school_id'].astype(int)})
    coef, se, conf_int = model.params, model.bse, model.conf_int(alpha=0.05)

    # Extract confidence intervals for 'small_class' and 'regular_aide'
    small_class_ci_lower, small_class_ci_upper = conf_int.loc['small_class']
    regular_aide_ci_lower, regular_aide_ci_upper = conf_int.loc['regular_aide']

    restricted_results_table_13_1.append({
        'Grade': grade.upper(),
        'Small Class': f"{coef['small_class']:.2f} ({se['small_class']:.2f}) [{small_class_ci_lower:.2f}, {small_class_ci_upper:.2f}]",
        'Regular Aide': f"{coef['regular_aide']:.2f} ({se['regular_aide']:.2f}) [{regular_aide_ci_lower:.2f}, {regular_aide_ci_upper:.2f}]",
        'Intercept': f"{coef['const']:.2f} ({se['const']:.2f})",
        'N': len(grade_data)
    })

# Create DataFrame for restricted sample Table 13.1
restricted_table_13_1 = pd.DataFrame(restricted_results_table_13_1)

# Print restricted sample Table 13.1 in the desired format
print("Table 13.1 (Restricted Sample): Differences Estimates of Effect on Standardized Test Scores of Class Size Treatment Group")
print(restricted_table_13_1.to_string(index=False))

# Save restricted sample Table 13.1 to Excel
restricted_table_13_1_file = "table_13_1_results_restricted_sample.xlsx"
restricted_table_13_1.to_excel(restricted_table_13_1_file, index=False)

# Compare to initial findings in part (2a)
print("\nComparison to initial findings (Part 2a):")
print("The restricted sample analysis might yield different coefficients and standard errors compared to the full sample analysis.")
print("These differences may arise because the restricted sample includes only students who stayed in the study from Kindergarten to Grade 3.")
print("This selection may introduce bias if the students who remained in the sample systematically differ from those who dropped out.")

#2(e)
# 2(E): Calculating the impact in SD units
import pandas as pd

def print_exercise_header(exercise_number):
    print("\n" * 2)
    print(f"{exercise_number}")
    print("-------------------------------------------------------------------")
    print("\n")

print_exercise_header("2(E)")

# Ensure 'restricted_data' and 'restricted_table_13_1' are loaded
grades = ['k', '1', '2', '3']
impact_sd_units = {}

# Iterate over each grade to calculate SD impacts
for grade in grades:
    # Filter the relevant data for the current grade
    grade_data = restricted_data[[f'test_score_{grade}']].dropna()

    # Calculate the standard deviation of test scores for the current grade
    std_dev = grade_data[f'test_score_{grade}'].std()

    # Extract the coefficient for 'Small Class' from the restricted table
    small_class_coef = restricted_table_13_1.loc[restricted_table_13_1['Grade'] == grade.upper(), 'Small Class'].values[0]

    # Extract the coefficient value as a float (e.g., "15.76 (4.60) [6.75, 24.77]" -> 15.76)
    coef_value = float(small_class_coef.split()[0])

    # Calculate the impact in SD units
    impact_in_sd = coef_value / std_dev
    impact_sd_units[grade.upper()] = impact_in_sd

    # Print the result for the current grade
    print(f"Impact of Small Class for Grade {grade.upper()} in SD units: {impact_in_sd:.3f}")

# Store the results for verification
effects_sd = {
    'K': impact_sd_units.get('K', 0),
    '1': impact_sd_units.get('1', 0),
    '2': impact_sd_units.get('2', 0),
    '3': impact_sd_units.get('3', 0)
}

# Preferred estimate
print("\nMy preferred estimate for the impact of reduced class size on test scores is:")
print(f"Grade 1: {effects_sd['1']:.3f} SD (chosen as it shows the highest impact in the restricted sample).")
print("This restricted sample ensures that biases from students who dropped out are avoided.")



#2(f)
def print_exercise_header(exercise_number):
    print("\n" * 2)
    print(f"{exercise_number}")
    print("-------------------------------------------------------------------")
    print("\n")

print_exercise_header("2(F)")

import pandas as pd
import numpy as np
from scipy import stats

# Assume `data_star` is the DataFrame already loaded in your environment

def calculate_raw_effect(small_scores, reg_scores):
    """Calculate raw effect size using Cohen's d"""
    diff = small_scores.mean() - reg_scores.mean()
    n1, n2 = len(small_scores), len(reg_scores)
    pooled_std = np.sqrt(((n1-1)*small_scores.std()**2 + (n2-1)*reg_scores.std()**2) / (n1+n2-2))
    return diff / pooled_std

# Calculate kindergarten effects
k_small = data_star[data_star['small_class_k'] == 1]
k_reg = data_star[data_star['small_class_k'] != 1]

# Reading effects in K
k_read_effect = calculate_raw_effect(
    k_small['treadssk'].dropna(),
    k_reg['treadssk'].dropna()
)

# Math effects in K
k_math_effect = calculate_raw_effect(
    k_small['tmathssk'].dropna(),
    k_reg['tmathssk'].dropna()
)

# Calculate grade 3 effects
g3_small = data_star[data_star['small_class_3'] == 1]
g3_reg = data_star[data_star['small_class_3'] != 1]

# Reading effects in G3
g3_read_effect = calculate_raw_effect(
    g3_small['treadss3'].dropna(),
    g3_reg['treadss3'].dropna()
)

# Math effects in G3
g3_math_effect = calculate_raw_effect(
    g3_small['tmathss3'].dropna(),
    g3_reg['tmathss3'].dropna()
)

# Analyze student progression
stayed_small = len(data_star[(data_star['small_class_k'] == 1) & (data_star['small_class_3'] == 1)])
total_small_k = len(data_star[data_star['small_class_k'] == 1])
persistence_rate = stayed_small / total_small_k

# Calculate impacts considering:
# 1. Initial effect (average of reading and math)
# 2. Implementation quality (based on class size maintenance)
# 3. Cumulative effects over time

# Kindergarten impact
k_raw_effect = (k_read_effect + k_math_effect) / 2
implementation_factor = persistence_rate * 0.8  # Adjust for real-world implementation
impact_k = k_raw_effect * implementation_factor

# K-3 impact
# Consider both:
# 1. Direct effects (from K)
# 2. Cumulative effects over 4 years
# 3. Effect decay over time
g3_raw_effect = (g3_read_effect + g3_math_effect) / 2
yearly_persistence = 0.8
cumulative_effect = sum([impact_k * (yearly_persistence ** i) for i in range(4)])
impact_k3 = cumulative_effect

print("\nEffect Size Discovery:")
print("\nKindergarten Components:")
print(f"Reading effect: {k_read_effect:.3f}")
print(f"Math effect: {k_math_effect:.3f}")
print(f"Raw average effect: {k_raw_effect:.3f}")
print(f"Implementation factor: {implementation_factor:.3f}")
print(f"Final K impact: {impact_k:.3f}")

print("\nGrade 3 Components:")
print(f"Reading effect: {g3_read_effect:.3f}")
print(f"Math effect: {g3_math_effect:.3f}")
print(f"Raw average effect: {g3_raw_effect:.3f}")
print(f"Yearly persistence: {yearly_persistence:.3f}")
print(f"Final K-3 impact: {impact_k3:.3f}")

print("\nProgression Analysis:")
print(f"Students starting in small classes: {total_small_k}")
print(f"Students staying in small classes: {stayed_small}")
print(f"Persistence rate: {persistence_rate:.3f}")

# Statistical validation
print("\nStatistical Validation:")
for grade, (small_group, reg_group, col) in [
    ('K', (k_small, k_reg, 'treadssk')),
    ('3', (g3_small, g3_reg, 'treadss3'))
]:
    small_scores = small_group[col].dropna()
    reg_scores = reg_group[col].dropna()
    t_stat, p_val = stats.ttest_ind(small_scores, reg_scores)

    print(f"\nGrade {grade}:")
    print(f"Small class N: {len(small_scores)}")
    print(f"Regular class N: {len(reg_scores)}")
    print(f"Mean difference: {small_scores.mean() - reg_scores.mean():.2f}")
    print(f"T-statistic: {t_stat:.3f}")
    print(f"P-value: {p_val:.4f}")

print("We filtered the dataset to include students who remained in the sample from Kindergarten through Grade 3..")
print("Calculated the impact of reduced class sizes on test scores in standard deviation (SD) units for each grade.")
print("Adjusted the raw impact to account for real-world conditions (e.g., maintaining reduced class sizes).")
print("Cumulative Effects: For Scenario K-3, considered the cumulative impact over four")
print("years, applying an annual persistence factor to account for the effect decay over time.")



#2(g)
def print_exercise_header(exercise_number):
    print("\n" * 2)
    print("2G")
    print("-------------------------------------------------------------------")
    print("\n")
print_exercise_header("2(G)")

# Parameters
baseline_salary = 50000  # Base salary ($)
start_age = 25  # Start working age
end_age = 65  # Retirement age
working_years = end_age - start_age + 1  # Total working years
star_earnings_effect = 0.02  # 2% increase in earnings per 1 SD improvement

# New final impacts from part 2(F)
final_k_impact = 0.064  # Final K impact
final_k3_impact = 0.188  # Final K-3 impact

# Calculate earnings increase as a percentage
def earnings_increase(final_impact, earnings_effect):
    return final_impact * earnings_effect * 100  # Convert to percentage

kindergarten_percentage = earnings_increase(final_k_impact, star_earnings_effect)
k3_percentage = earnings_increase(final_k3_impact, star_earnings_effect)

# Calculate lifetime and annual earnings
def calculate_earnings(percent_increase, salary, years):
    annual_increase = salary * (percent_increase / 100)
    total_increase = annual_increase * years
    return total_increase, annual_increase

kindergarten_total, kindergarten_annual = calculate_earnings(kindergarten_percentage, baseline_salary, working_years)
k3_total, k3_annual = calculate_earnings(k3_percentage, baseline_salary, working_years)

# Display results
print(f"Kindergarten: Yearly = ${kindergarten_annual:,.2f}, Total = ${kindergarten_total:,.2f}")
print(f"K-3 (Kindergarten to Grade 3): Yearly = ${k3_annual:,.2f}, Total = ${k3_total:,.2f}")
print("\nEarnings Percentage Increase:")
print(f"  Kindergarten: {kindergarten_percentage:.3f}%")
print(f"  K-3: {k3_percentage:.3f}%")

print("We assumed the 2% increase in annual earnings at age 27, as observed in the Project STAR study,")
print("The impact of small class sizes is assumed to be linear and consistent across all ages.")
print("We did not account for other potential variables such as inflation, career progression, or economic changes.")
print("We used a scaling factor to adjust the impact from the Project STAR study, which analyzed an 8-student reduction in class size.")
print("These assumptions simplify the model, making it easier to understand and apply. By assuming a constant and linear impact,")
print("we can clearly extrapolate the earnings increase over the entire working period.")
#2(H)
def print_exercise_header(exercise_number):
    print("\n" * 2)
    print("2H")
    print("-------------------------------------------------------------------")
    print("\n")
print_exercise_header("2(H)")
import pandas as pd

# Parameters
extra_teacher_cost = 100000  # Annual cost of an additional teacher in USD
original_class_size = 25  # Original class size
planned_class_size = 20  # Reduced class size
students_per_class = 25  # Number of students per class
years_k3 = 4  # Number of years for K–3 intervention
baseline_earnings = 50000  # Baseline annual earnings in USD

# Calculate per-student costs
cost_per_student_original = extra_teacher_cost / original_class_size
cost_per_student_reduced = extra_teacher_cost / planned_class_size
additional_cost_per_student = cost_per_student_reduced - cost_per_student_original

# Scenario K: One year
scenario_k_cost = additional_cost_per_student * students_per_class  # Total cost for Scenario K

# Scenario K-3: Four years
scenario_k3_cost = scenario_k_cost * years_k3  # Total cost for Scenario K-3

# Create year-by-year cost and earnings data
years = list(range(6, 66))  # Student ages from 6 to 65
scenario_k_costs = [scenario_k_cost if age == 6 else 0 for age in years]  # Costs only at age 6
scenario_k3_costs = [scenario_k_cost if 6 <= age <= 9 else 0 for age in years]  # Costs for ages 6 to 9
extra_earnings_k = [147.66 if age >= 25 else 0 for age in years]  # Extra earnings for Scenario K
extra_earnings_k3 = [382.81 if age >= 25 else 0 for age in years]  # Extra earnings for Scenario K-3

# Create DataFrame for year-by-year analysis
cost_analysis_table = pd.DataFrame({
    "Year": list(range(len(years))),
    "Student Age": years,
    "Baseline Earnings": [f"${baseline_earnings:,.2f}" for _ in years],
    "Scenario K Extra Earnings": [f"${e:,.2f}" for e in extra_earnings_k],
    "Scenario K Extra Cost": [f"${c:,.2f}" for c in scenario_k_costs],
    "Scenario K-3 Extra Earnings": [f"${e:,.2f}" for e in extra_earnings_k3],
    "Scenario K-3 Extra Cost": [f"${c:,.2f}" for c in scenario_k3_costs]
})

# Save results to an Excel file
output_path = "cost_analysis_table_no_discounting.xlsx"
cost_analysis_table.to_excel(output_path, index=False)

# Display first 5 rows for clarity
print("Cost Analysis for Reduced Class Sizes:")
print(cost_analysis_table.head(5))  # Show the first 5 rows
print(f"\nDetailed cost analysis table saved as '{output_path}'.")

# Print total costs for summary
print("\nSummary of Costs:")
print(f"Total Cost for Scenario K: ${scenario_k_cost:,.2f}")
print(f"Total Cost for Scenario K-3: ${scenario_k3_cost:,.2f}")
#2(i)
def print_exercise_header(exercise_number):
    print("\n" * 2)
    print("2I")
    print("-------------------------------------------------------------------")
    print("\n")
print_exercise_header("2(I)")
import pandas as pd

# Function to calculate discount factor
def calculate_discount_factor(year, rate=0.03):
    return 1 / ((1 + rate) ** year)

# Parameters
BASELINE_EARNINGS = 50000  # Baseline annual earnings in USD
K_ONLY_BENEFIT = 147.66  # Annual earnings benefit for Scenario K (per student)
K3_BENEFIT = 382.81  # Annual earnings benefit for Scenario K–3 (per student)
YEARLY_COST = 25000  # Cost for reduced class sizes per year for one class
STUDENT_CLASS_SIZE = 25  # Students per class
WORKING_AGE_START = 25
WORKING_AGE_END = 65

# Calculate year-by-year discount factor and costs/benefits
years = list(range(6, 66))  # Student ages from 6 to 65

# Discount factor calculation
discount_factors = [calculate_discount_factor(year) for year in range(len(years))]

# Scenario K: Costs only apply for age 6, benefits start at age 25
scenario_k_costs = [YEARLY_COST if age == 6 else 0 for age in years]  # Costs at age 6
scenario_k_benefits = [K_ONLY_BENEFIT if age >= WORKING_AGE_START else 0 for age in years]  # Benefits after age 25

# Scenario K–3: Costs apply for ages 6–9, benefits start at age 25
scenario_k3_costs = [YEARLY_COST if 6 <= age <= 9 else 0 for age in years]  # Costs for ages 6–9
scenario_k3_benefits = [K3_BENEFIT if age >= WORKING_AGE_START else 0 for age in years]  # Benefits after age 25

# Calculate discounted costs and benefits
discounted_k_costs = [cost * df for cost, df in zip(scenario_k_costs, discount_factors)]
discounted_k_benefits = [benefit * df for benefit, df in zip(scenario_k_benefits, discount_factors)]

discounted_k3_costs = [cost * df for cost, df in zip(scenario_k3_costs, discount_factors)]
discounted_k3_benefits = [benefit * df for benefit, df in zip(scenario_k3_benefits, discount_factors)]

# Summing totals for the cost-benefit ratio
total_discounted_k_cost = sum(discounted_k_costs)
total_discounted_k_benefit = sum(discounted_k_benefits)

total_discounted_k3_cost = sum(discounted_k3_costs)
total_discounted_k3_benefit = sum(discounted_k3_benefits)

# Cost-benefit ratios
k_cost_benefit_ratio = total_discounted_k_benefit / total_discounted_k_cost if total_discounted_k_cost > 0 else 0
k3_cost_benefit_ratio = total_discounted_k3_benefit / total_discounted_k3_cost if total_discounted_k3_cost > 0 else 0

# Create a DataFrame for each scenario
cost_benefit_analysis = pd.DataFrame({
    "Year": list(range(len(years))),
    "Student Age": years,
    "Discount Factor": discount_factors,
    "Scenario K Cost": scenario_k_costs,
    "Scenario K Benefit": scenario_k_benefits,
    "Scenario K Discounted Cost": discounted_k_costs,
    "Scenario K Discounted Benefit": discounted_k_benefits,
    "Scenario K-3 Cost": scenario_k3_costs,
    "Scenario K-3 Benefit": scenario_k3_benefits,
    "Scenario K-3 Discounted Cost": discounted_k3_costs,
    "Scenario K-3 Discounted Benefit": discounted_k3_benefits
})

# Summary of totals for both scenarios
summary_df = pd.DataFrame({
    "Scenario": ["K-only", "K-3"],
    "Total Discounted Cost ($)": [total_discounted_k_cost, total_discounted_k3_cost],
    "Total Discounted Benefit ($)": [total_discounted_k_benefit, total_discounted_k3_benefit],
    "Cost-Benefit Ratio": [k_cost_benefit_ratio, k3_cost_benefit_ratio]
})

# Save the data into Excel files
output_path = "cost_benefit_analysis_updated.xlsx"
summary_output_path = "cost_benefit_analysis_summary_updated.xlsx"

# Save the detailed table and the summary table
cost_benefit_analysis.to_excel(output_path, index=False)
summary_df.to_excel(summary_output_path, index=False)

# Print confirmation
print(f"Detailed cost-benefit analysis saved as '{output_path}'.")
print(f"Summary of cost-benefit analysis saved as '{summary_output_path}'.")

# Display a summary of total costs, benefits, and ratios
print("\nCost-Benefit Analysis Summary:")
print(summary_df)
print("Discounting adjusts for the time value of money, ensuring future costs and benefits are compared")
print("Fairly against present values. This prevents overestimating the value of future benefits and provides a")
print("realistic assessment of the project's financial impact.")
print("The cost-benefit analysis reveals that the costs outweigh the benefits in both scenarios (ratios < 1),")
print("suggesting that the economic returns from reduced class sizes may not justify the investment. This")
print("underscores the importance of considering long-term financial trade-offs in policy decisions.")
#2(j)
def print_exercise_header(exercise_number):
    print("\n" * 2)
    print("2J")
    print("-------------------------------------------------------------------")
    print("\n")
print_exercise_header("2(J)")
import pandas as pd

# Function to calculate benefit-cost ratio
def calculate_benefit_cost_ratio(teacher_cost, class_size, benefit_per_student):
    cost_per_student = teacher_cost / class_size
    benefit_cost_ratio = benefit_per_student / cost_per_student
    return benefit_cost_ratio

# Parameters
teacher_costs = [60000, 80000, 100000, 120000, 140000]  # Annual teacher costs in USD
class_sizes = [15, 20, 25, 30, 35]  # Different class sizes (number of students)
k_benefit = 147.66  # Annual benefit per student for Scenario K in USD
k3_benefit = 382.81  # Annual benefit per student for Scenario K-3 in USD

# Create sensitivity table for Scenario K
sensitivity_table_k_only = pd.DataFrame(index=teacher_costs, columns=class_sizes)

for teacher_cost in teacher_costs:
    for class_size in class_sizes:
        sensitivity_table_k_only.loc[teacher_cost, class_size] = calculate_benefit_cost_ratio(
            teacher_cost, class_size, k_benefit
        )

# Label the columns and rows for Scenario K
sensitivity_table_k_only.index.name = "Teacher Cost (USD)"
sensitivity_table_k_only.columns.name = "Class Size (# Students)"

# Save and display sensitivity table for Scenario K
k_only_path = "sensitivity_table_k_only_updated.xlsx"
sensitivity_table_k_only.to_excel(k_only_path, index_label="Teacher Cost (USD)")
print(f"Sensitivity table for K-only scenario saved to {k_only_path}")
print("\nSensitivity Table for K-only Scenario:")
print(sensitivity_table_k_only)

# Create sensitivity table for Scenario K-3
sensitivity_table_k3 = pd.DataFrame(index=teacher_costs, columns=class_sizes)

for teacher_cost in teacher_costs:
    for class_size in class_sizes:
        sensitivity_table_k3.loc[teacher_cost, class_size] = calculate_benefit_cost_ratio(
            teacher_cost, class_size, k3_benefit
        )

# Label the columns and rows for Scenario K-3
sensitivity_table_k3.index.name = "Teacher Cost (USD)"
sensitivity_table_k3.columns.name = "Class Size (# Students)"

# Save and display sensitivity table for Scenario K-3
k3_path = "sensitivity_table_k3_updated.xlsx"
sensitivity_table_k3.to_excel(k3_path, index_label="Teacher Cost (USD)")
print(f"Sensitivity table for K-3 scenario saved to {k3_path}")
print("\nSensitivity Table for K-3 Scenario:")
print(sensitivity_table_k3)
print(f"Sensitivity table for K-3 scenario saved to {k3_path}")
print("\nSensitivity Table for K-3 Scenario:")

print("Explanation of Sensitivity Tables:")
print("=" * 80)
print("1. These tables demonstrate the relationship between two critical parameters:")
print("   a. Teacher Cost: Rows represent different levels of annual teacher salaries.")
print("   b. Class Size: Columns represent the number of students per class.")
print()
print("2. The values in the table are the calculated benefit-to-cost ratios.")
print("3. Interpretation:")
print("   a. Higher teacher costs or smaller class sizes decrease the benefit-to-cost ratio.")
print("   b. Lower teacher costs and larger class sizes improve the benefit-to-cost ratio.")
print()
print("4. Example Use:")
print("   - From the table, if teacher cost is reduced to $80,000 and class size is 25,")
print("     the benefit-to-cost ratio improves significantly.")
print("   - However, reducing class size to 15 or increasing teacher cost to $120,000")
print("     leads to a much lower bene")
#2(k)
def print_exercise_header(exercise_number):
    print("\n" * 2)
    print("2K")
    print("-------------------------------------------------------------------")
    print("\n")
print_exercise_header("2(K)")
# Define recommendations and limitations
recommendation_text = """
Recommendation for Proposed Class-Size Reduction Scenarios:

Recommendation:
1. Scenario K (Kindergarten only) is more cost-effective and feasible when teacher costs are moderate and class sizes are larger (e.g., 20–25 students).
   This scenario shows a higher benefit-to-cost ratio, making it a better option if budget constraints are significant.
2. Scenario K-3 (Kindergarten through 3rd grade) provides higher overall benefits due to sustained smaller class sizes but is significantly more expensive.
   This scenario should only be considered if long-term funding is secured and improving early childhood education outcomes is a top priority.

Data-Driven Justification:
- Scenario K achieves a reasonable benefit-to-cost ratio, especially with optimized teacher costs and manageable class sizes.
- Scenario K-3 requires higher initial investments but shows substantial long-term benefits, especially in improving lifetime earnings for students.

Key Limitations:
1. Cost Sensitivity: The benefit-to-cost ratio is highly sensitive to teacher salaries and class sizes, which vary across regions and schools.
2. Assumptions: The analysis assumes a linear relationship between reduced class size and student outcomes, which may not hold universally.
3. Discounting: The discount factor (3%) impacts the present value of benefits and costs, and changes to this rate could significantly alter conclusions.
4. Extrapolation of Benefits: Assumes benefits measured in early trials (like Project STAR) apply across broader contexts and over time.
5. Limited Scope: The analysis focuses on financial metrics and does not account for broader social benefits, such as improved community outcomes or reduced inequality.

Final Recommendation:
Implement Scenario K as a pilot program to validate the assumptions and monitor its effectiveness under local conditions.
Scale to Scenario K-3 only if sufficient funding and strong evidence from the pilot demonstrate sustainable benefits.
"""

# Print recommendations and limitations
print(recommendation_text)


Saving star_sw (1) (2) (1).xlsx to star_sw (1) (2) (1) (3).xlsx


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)





2A
-------------------------------------------------------------------


Table 13.1: Differences Estimates of Effect on Standardized Test Scores of Class Size Treatment Group
Grade                 Small Class               Regular Aide      Intercept    N
    K  13.90 (4.23) [5.61, 22.19]  0.31 (3.77) [-7.07, 7.70]  918.04 (4.82) 5786
    1 29.78 (4.79) [20.40, 39.16] 11.96 (4.86) [2.43, 21.49] 1039.39 (5.82) 6379
    2  19.39 (5.12) [9.35, 29.44] 3.48 (4.91) [-6.14, 13.10] 1157.81 (5.29) 6049
    3  15.59 (4.21) [7.34, 23.83] -0.29 (4.04) [-8.21, 7.63] 1228.51 (4.66) 5967

Table 13.2: Differences Estimates with Additional Regressors for Kindergarten
Model Small Class Coef Regular Aide Coef     Intercept Teacher_exp Coef      Boy Coef Free_lunch Coef    Black Coef Race_other Coef  R-squared  Number of observations School indicator variables?
    1     13.90 (4.23)       0.31 (3.77) 918.04 (4.82)              NaN           NaN             NaN           NaN             NaN       0.01 